In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python import keras as kt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [57]:

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


X_test = np.swapaxes(X_test, 1, 2)
print(X_test.shape)
X_test = X_test[: , : , :22 ]

print(X_test.shape)

(443, 1000, 25)
(443, 1000, 22)


### Shape of data

In [58]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [59]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN, SimpleRNN
from keras.utils import to_categorical
y_binary = to_categorical(y_test-769)
hidden_size = 100

In [62]:

model = Sequential()
model.add(SimpleRNN(hidden_size, input_shape=(1000, 22)))
model.add(Dense(4, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'],
             )


model.fit(X_test, y_binary, epochs=10)

Epoch 1/10
443/443 [==============================] - 4s 9ms/step - loss: 1.6943 - acc: 0.2506
Epoch 2/10
443/443 [==============================] - 3s 6ms/step - loss: 1.4053 - acc: 0.3363
Epoch 3/10
443/443 [==============================] - 3s 6ms/step - loss: 1.3188 - acc: 0.3860
Epoch 4/10
443/443 [==============================] - 3s 6ms/step - loss: 1.2459 - acc: 0.4176
Epoch 5/10
443/443 [==============================] - 3s 6ms/step - loss: 1.1977 - acc: 0.4831
Epoch 6/10
443/443 [==============================] - 3s 6ms/step - loss: 1.1579 - acc: 0.5237
Epoch 7/10
443/443 [==============================] - 3s 6ms/step - loss: 1.1217 - acc: 0.5418
Epoch 8/10
443/443 [==============================] - 3s 6ms/step - loss: 1.0862 - acc: 0.5598
Epoch 9/10
443/443 [==============================] - 3s 6ms/step - loss: 1.0601 - acc: 0.6027
Epoch 10/10
443/443 [==============================] - 3s 6ms/step - loss: 1.0299 - acc: 0.6230
